In [ ]:
!pip install openai tiktoken chromadb langchain

In [ ]:
import dotenv
import os

from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

In [ ]:
dotenv.load_dotenv()

api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    GenParams.DECODING_METHOD:"greedy",
    GenParams.MAX_NEW_TOKENS:300,
    GenParams.MIN_NEW_TOKENS:15,
    GenParams.REPETITION_PENALTY:2
}

In [ ]:
model = Model(ModelTypes.LLAMA_2_70B_CHAT,creds,params,project_id)
llm = WatsonxLLM(model)

In [ ]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

In [ ]:
loader = GenericLoader.from_filesystem(
    "../../samples/",
    glob="**/*",
    suffixes=[".sql"],
    parser=LanguageParser(language=Language.SOL, parser_threshold=500)
)
documents = loader.load()
len(documents)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# sql_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.SQL, 
#                                                                chunk_size=2000, 
#                                                                chunk_overlap=200)
sql_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,chunk_overlap=200, separators=["\n\n","\n","select"])
texts = sql_splitter.split_documents(documents)
len(texts)
for text in texts:
    print(text)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceInstructEmbeddings
db = Chroma.from_documents(texts, HuggingFaceInstructEmbeddings())
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
question = "Generate a SQL to create Table with 2 column: Name, Id"
result = qa(question)
result['answer']

In [ ]:
questions = [
    "What is the class hierarchy?",
    "What classes are derived from the Chain class?",
    "What one improvement do you propose in code in relation to the class herarchy for the Chain class?",
]

for question in questions:
    result = qa(question)
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")